In [ ]:
import argparse
import sys
import time
from pathlib import Path

import torch
import torch.nn as nn
from torch.utils.mobile_optimizer import optimize_for_mobile
import cv2
import numpy as np

import models
from models.experimental import attempt_load
from utils.activations import Hardswish, SiLU
from utils.general import colorstr, check_img_size, check_requirements, file_size, set_logging
from utils.torch_utils import select_device

## First step load model

In [ ]:
device = 'cuda:0'

model = attempt_load('./yolov7-tiny-face.pt', map_location=device)
for k, m in model.named_modules():
    m._non_persistent_buffers_set = set()  # pytorch 1.6.0 compatibility
    if isinstance(m, models.common.Conv):  # assign export-friendly activations
        if isinstance(m.act, nn.Hardswish):
            m.act = Hardswish()
        elif isinstance(m.act, nn.SiLU):
            m.act = SiLU()
    if isinstance(m, models.common.Concat):
        m.forward = m.forward_jit
        
    # elif isinstance(m, models.yolo.Detect):
    #     m.forward = m.forward_export  # assign forward (optional)
model.model[-1].export = False
model = model.eval()

## Setting torch_tensorrt

In [ ]:
arr = np.arange(10)
np.random.shuffle(arr)
print(arr)
arr.argsort()[::-1]